# **Import dependencies**

In [ ]:
import os

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from flasc.dataframe_operations import (
    dataframe_filtering as dff,
    dataframe_manipulations as dfm,
)
from flasc.turbine_analysis import ws_pow_filtering as wspf

from {{cookiecutter.project_slug}}.models import load_floris

In [ ]:
# User settings
save_figures = True
plot_figures_in_notebook = True

# **Step 0**: Initial data pulldown
First, we import the data from the common_windfarm_information folder. This may take a while, so we keep these variables unchanged. These are df_scada_raw and df_metmast_raw. These variables are not manipulated throughout the script.

In [ ]:
def load_data():
    root_path = os.getcwd()
    source_path = os.path.join(root_path, "..", "..", "common_windfarm_information")
    df_scada_raw = pd.read_csv(
        os.path.join(source_path, "demo_dataset_scada_600s.csv")
    )
    df_metmast_raw = pd.read_csv(
        os.path.join(source_path, "demo_dataset_metmast_600s.csv")
    )

    # Drop first and "empty" column with indices
    df_scada_raw = df_scada_raw.drop(df_scada_raw.columns[0], axis=1)
    df_metmast_raw = df_metmast_raw.drop(df_metmast_raw.columns[0], axis=1)

    print("Columns available in df_scada_raw: {}.".format(list(df_scada_raw.columns)))
    return df_scada_raw, df_metmast_raw

df_scada_raw, df_metmast_raw = load_data()

# **Step 1**: Format to common FLASC format
Now create a copy of df_scada_raw and df_metmast_raw which we can manipulate and filter.

In [ ]:
# Now make a copy of the raw data files for processing and manipulation
df_scada = df_scada_raw.copy()
df_metmast = df_metmast_raw.copy()

Format df_scada to pour the dataframe into the common FLASC format. For example, wind speeds are columns denoted by ws_{ti}, with {ti} the turbine number with prevailing zeros. Hence, for wind speed for the third turbine is defined by ws_002, and the power production of the thirteenth turbine is defined by pow_012.

In [ ]:
def format_dataframes(df_scada, df_metmast):
    # Format columns and data. The operations required differ per dataset.
    df_scada["time"] = pd.to_datetime(df_scada["time"])  # Convert strings to timestamps
    df_metmast["time"] = pd.to_datetime(df_metmast["time"])  # Convert strings to timestamps

    # In FLORIS, turbines are numbered from 0 to nturbs - 1. In SCADA data,
    # turbines often have a different name. We save the mapping between
    # the turbine indices in FLORIS and the turbine names to a separate .csv
    # file.
    root_path = os.getcwd()
    out_path = os.path.join(root_path, "postprocessed")
    os.makedirs(out_path, exist_ok=True)
    turbine_names = ["A1", "A2", "A3", "B1", "B2", "C1", "C2"]
    pd.DataFrame({"turbine_names": turbine_names}).to_csv(
        os.path.join(out_path, "turbine_names.csv")
    )

    # Now map columns to conventional format
    scada_dict = {}
    for ii, tn in enumerate(turbine_names):
        scada_dict.update(
            {
                "ActivePower_{:s}".format(tn): "pow_{:03d}".format(ii),
                "NacWSpeed_{:s}".format(tn): "ws_{:03d}".format(ii),
                "NacTI_{:s}".format(tn): "ti_{:03d}".format(ii),
                "NacWDir_{:s}".format(tn): "wd_{:03d}".format(ii),
                "is_operation_normal_{:s}".format(tn): "is_operation_normal_{:03d}".format(ii),
            }
        )

    df_list = []
    print("formatting dataframe...")
    df_scada = df_scada.rename(columns=scada_dict)

    # Reduce precision in dataframe to use half of the memory
    df_scada = dfm.df_reduce_precision(df_scada, verbose=True)
    df_metmast = dfm.df_reduce_precision(df_metmast, verbose=True)

    # Sort dataframe and save
    df_scada = df_scada.sort_values(axis=0, by="time")
    df_scada = df_scada.reset_index(drop=True)
    print("Columns available in df_scada: {}.".format(list(df_scada.columns)))

    return df_scada, df_metmast

print(df_scada.columns)
df_scada, df_metmast = format_dataframes(df_scada, df_metmast)

# **Step 2**: Filter the data for outliers
We apply a number of filtering operations here, like the removal of NaN wind speed/power measurements, negative wind speed/power measurements, irregularly high wind speed/power measurements, sensor-stuck type of faults, self-flagged faults (i.e., internal turbine status flags), and filtering based on deviations from the median windspeed-power curve.

In [ ]:
# # **Step 4**: Deal with wind-speed power curve filtering
def filter_by_ws_pow_curve(df, plot_figures=True, save_figures=False):
    # Load the FLORIS model for the wind farm. This is not used for anything
    # besides plotting the floris-predicted wind speed-power curve on top
    # of the actual data.
    fi = load_floris()

    # Downsample data. Not necessary here, but can be useful if we have 1 Hz
    # data available. Namely, it's hard to detect outliers on such a high
    # resolution. Instead, we are better off downsampling the data to 60s or
    # even 600s and filter the data based on decisions there. The following
    # downsampled dataframe should then be inserted into the wind speed power
    # curve filtering class. Mapping the filtering back to the high-resolution
    # data is done by a couple lines of code as found at the end of this
    # script.
    #
    # df_movavg, data_indices_mapping = top.df_movingaverage(
    #     df_in=df_1s,
    #     cols_angular=[
    #         c for c in df_1s.columns if (
    #             ("vane_" in c) or
    #             ("yaw_" in c) or
    #             ("wd_" in c) or
    #             ("direction" in c)
    #         )
    #     ],
    #     window_width=td(seconds=600),
    #     calc_median_min_max_std=False,
    #     return_index_mapping=True,
    # )

    # Create output directory
    if (plot_figures) & (save_figures):
        root_path = os.getcwd()
        figure_save_path = os.path.join(root_path, "postprocessed", "figures", "04_wspowcurve_filtered")
        os.makedirs(figure_save_path, exist_ok=True)
    else:
        figure_save_path = None

    # Initialize the wind speed power curve filtering class
    ws_pow_filtering = wspf.ws_pw_curve_filtering(df=df)

    # Apply a set of logic filters on the turbine measurements
    n_turbines = dfm.get_num_turbines(df)
    for ti in range(n_turbines):
        # Filter for NaN wind speed or power productions
        ws_pow_filtering.filter_by_condition(
            condition=(
            ws_pow_filtering.df["ws_{:03d}".format(ti)].isna() |
            ws_pow_filtering.df["pow_{:03d}".format(ti)].isna()
            ),
            label="Wind speed and/or power is NaN",
            ti=ti,
            verbose=True,
        )

        # Filter for numerical issues
        ws_pow_filtering.filter_by_condition(
            condition=(ws_pow_filtering.df["ws_{:03d}".format(ti)] < -1.0e-6),
            label="Wind speed below zero",
            ti=ti,
            verbose=True,
        )

        # Filter for numerical issues
        ws_pow_filtering.filter_by_condition(
            condition=(ws_pow_filtering.df["pow_{:03d}".format(ti)] < -1.0e-6),
            label="Power below zero",
            ti=ti,
            verbose=True,
        )

        # Filter for numerical issues
        ws_pow_filtering.filter_by_condition(
            condition=(ws_pow_filtering.df["ws_{:03d}".format(ti)] > 50),
            label="Wind speed above 50 m/s",
            ti=ti,
            verbose=True,
        )

        # Filter for numerical issues: note, make sure power is in kW
        ws_pow_filtering.filter_by_condition(
            condition=(ws_pow_filtering.df["pow_{:03d}".format(ti)] > 30e3),
            label="Power above 30 MW",
            ti=ti,
            verbose=True,
        )

        # Filter for power production is zero above cut-in wind speeds
        ws_pow_filtering.filter_by_condition(
            condition=(ws_pow_filtering.df["ws_{:03d}".format(ti)] > 4.0) & (ws_pow_filtering.df["pow_{:03d}".format(ti)] < 1.0),
            label="Power below 1 kW while wind speed above 4 m/s",
            ti=ti,
            verbose=True,
        )

        # Other common filters here are based on turbine-specific/OEM-specific flags. For example,
        # the 'run counter' in Vestas turbines indicates how many seconds of a 10-minute period
        # the turbine was reporting valid measurements. By filtering for the condition that
        # that value is 600, or above 590, you should be able to quickly identify a large part
        # of faulty measurements. 
        #
        # Additionally, if you have blade pitch angle measurements, you may be able to filter
        # for outliers by looking when blade pitch angles exceed, e.g., 70 degrees. These filter
        # steps very much depend on the dataset that you have access to.
        # 
        # In this case, we have an operational_status flag. If that has a 'False' value, then we
        # mark those measurements as faulty.
        ws_pow_filtering.filter_by_condition(
            condition=(ws_pow_filtering.df["is_operation_normal_{:03d}".format(ti)] == False),
            label="Self-flagged (is_operation_normal==False)",
            ti=ti,
            verbose=True,
        )

        # Filter for sensor-stuck faults
        ws_pow_filtering.filter_by_sensor_stuck_faults(
            columns=["wd_{:03d}".format(ti), "ws_{:03d}".format(ti)],
            ti=ti,
            n_consecutive_measurements=3,
            stddev_threshold=0.001,
            plot=False,
        )

        # Flag curtailment by marking measurements with a high wind speed but
        # lower power production as faulty.
        ws_pow_filtering.filter_by_condition(
            condition=(
                (ws_pow_filtering.df["ws_{:03d}".format(ti)] > 10.2) &
                (ws_pow_filtering.df["pow_{:03d}".format(ti)] < 3200.0)
            ),
            label="Curtailment: wind speed above 10.2 m/s but power below 3200 kW",
            ti=ti,
            verbose=True,
        )

        # Now filter iteratively by deviations from the median power curve
        ws_pow_filtering.filter_by_power_curve(
            ti=ti,
            ws_deadband=1.5,
            pow_deadband=70.0,
            cutoff_ws=20.0,
            m_pow_rb=0.97,
        )

        # Plot and save data for current dataframe
        ws_pow_filtering.plot_filters_in_ws_power_curve(ti=ti, fi=fi)
        ws_pow_filtering.plot_filters_in_time(ti=ti)
        ws_pow_filtering.plot_postprocessed_in_ws_power_curve(ti=ti, fi=fi)
        print("\n")

    # Plot farm-averaged power curve based on the data (useful to e.g., feed into FLORIS when not provided by OEM)
    ws_pow_filtering.plot_farm_mean_power_curve()

    # Get filtered dataframe and power curve
    df = ws_pow_filtering.get_df()
    df_pow_curve = ws_pow_filtering.pw_curve_df

    return df, df_pow_curve


df_scada, df_pow_curve = filter_by_ws_pow_curve(df=df_scada)

# **Step 5**: Plot faults vs. the layout

In [ ]:
def plot_faults_vs_layout(df):
    fi = load_floris()

    layout_x = fi.layout_x
    layout_y = fi.layout_y

    num_turbines = dfm.get_num_turbines(df)
    fault_ratio = np.zeros(num_turbines)
    for ti in range(num_turbines):
        fault_ratio[ti] = (
            dff.df_get_no_faulty_measurements(df, ti) / df.shape[0]
        )

    # Plot layout and colormap
    fig, ax = plt.subplots(figsize=(14, 5))
    for ti in range(num_turbines):
        clr = [fault_ratio[ti], 1.0 - fault_ratio[ti], 0.0]
        ax.plot(
            layout_x[ti],
            layout_y[ti],
            "o",
            markersize=15,
            markerfacecolor=clr,
            markeredgewidth=0.0,
        )
        ax.text(
            layout_x[ti] + 100,
            layout_y[ti],
            "T%03d (%.1f%%)" % (ti, (1.0 - fault_ratio[ti]) * 100.0),
            color="black",
        )
    fig.tight_layout()

    root_path = os.getcwd()
    out_path = os.path.join(
        root_path,
        "data",
        "05_preliminary_fault_analysis",
        "show_filtered_faults_by_layout",
    )
    fig_out = os.path.join(out_path, "faults_by_layout.png")
    print("Saving figure to {:s}.".format(fig_out))
    os.makedirs(out_path, exist_ok=True)
    plt.savefig(fig_out, dpi=300)

plot_faults_vs_layout(df_scada)

# **Save postprocessed data** to a local file

In [ ]:
root_path = os.getcwd()
fout = os.path.join(root_path, "postprocessed", "df_scada_600s_wspowfiltered.ftr")
df_scada.to_feather(fout)
print("File saved to '{:s}'.".format(os.path.relpath(fout)))

In [ ]:
# Print all (remaining) figures
plt.show()